In [10]:
!pip install gdown

In [2]:
!pip install pandas

In [3]:
!pip install matplotlib

In [4]:
!pip install japanize_matplotlib

In [5]:
!pip install scikit-learn

In [11]:
import pandas as pd
import japanize_matplotlib

# ファイルパスを定義します
file_path = "weather/57066-2021_2023.csv"

# スキップする列を定義します
#read_columns = [0,1,4,7,10,13,16,19,21,24,27]
read_columns = [0,1,4,7,10,14,18,21,23,26,29]

# CSVファイルを読み込みます
df = pd.read_csv(file_path, encoding="cp932", skiprows=3, usecols=read_columns)[2:].reset_index(drop=True)
df.rename(columns={df.columns[8]: "風向"},inplace=True)
# 風向の列名を変更します
#df.rename(columns={df.columns[18]: "風向"}, inplace=True)

# DataFrameの最初の5行を表示します
#print(df.describe())
#df.head()
df.iloc[:5]

UnicodeDecodeError: 'cp932' codec can't decode byte 0xef in position 0: illegal multibyte sequence

In [12]:
# df["年月日"] の形式 yyyy/mm/dd　を yyyy で分割して、"年" 列を追加します
df["年"] = df["年月日"].str.split("/", expand=True)[0]

# df["年月日"] の形式 yyyy/mm/dd　を mm で分割して、"月" 列を追加します
df["月"] = df["年月日"].str.split("/", expand=True)[1]

# df["年月日"] の形式 yyyy/mm/dd　を dd で分割して、"日" 列を追加します
df["日"] = df["年月日"].str.split("/", expand=True)[2]

## 年月日の列を削除します
df.drop("年月日", axis=1, inplace=True)



NameError: name 'df' is not defined

In [ ]:
# 最後の3列を最初の3列に移動します
df = df[df.columns[-3:].tolist() + df.columns[:-3].tolist()]
df.head()


In [ ]:
# 列名を変更
new_columns = list(df.columns)[:3] + [col.split("(")[0] for col in df.columns[3:]]

# 列名の変更
df.columns = new_columns

df.head()


In [ ]:
import matplotlib.pyplot as plt

# 可視化したい月を指定
month = 8  # 1月は1

# 年リストを作成
years = df["年"].unique()

# 各項目を別々に描画
for col in df.columns[3:]:
    # 年別にデータを取得
    year_data = {}
    for year in years:
        year_data[year] = df[(df["月"] == str(month)) & (df["年"]==year)][col].to_list()
    # グラフ描画
    plt.figure(figsize=(10, 6))
    for year, data in year_data.items():
        plt.plot(range(1,len(data)+1),data, label=f"{year}年")
        plt.title(f"{col} ({month}月)")
        plt.xlabel("日")
        plt.ylabel(col)
        plt.legend()
    plt.show()



In [ ]:
#df[3:6].corr( )

# 数値データのみを抽出する
numeric_df = df.apply(pd.to_numeric, errors='coerce')

# 相関行列を計算する
correlation_matrix = numeric_df.corr()

numeric_df.corr()

In [ ]:
# 回帰分析に必要なライブラリをインポート
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
# データを準備
X = df[feature_cols]
y = df[target_col]

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# モデルを定義
model = LinearRegression()

# モデルを訓練
model.fit(X_train, y_train)

# テストデータで予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
print(f"平均二乗誤差: {mse}")


In [ ]:
model.coef_